### Traittement 

In [15]:
import json
import pandas as pd
from pydantic import BaseModel, ValidationError
from typing import Optional
from pydantic import BaseModel, field_validator
from typing import Optional

# Définition du modèle Pydantic pour valider les données
class Record(BaseModel):
    date: str  # Champ obligatoire
    nom: str   # Champ obligatoire
    ville: Optional[str]
    code: Optional[str]

    @field_validator('code')
    def code_must_start_with_qt(cls, value: Optional[str]) -> Optional[str]:
        if value is not None and not value.startswith('qt'):
            raise ValueError('Le code doit commencer par "qt"')
        return value

# Dictionnaire de mappage
mapping = {'1': 'date', '2': 'nom', '3': 'ville', '4': 'code'}

# Listes pour stocker les données valides et invalides
valid_records = []
invalid_records = []

# Lire le fichier .cdr
try:
    with open('sample.cdr', 'r') as file:
        # Supposons que chaque ligne contient un JSON
        for line in file:
            try:
                # Charger le JSON
                record = json.loads(line.strip())

                mapped_record = {}
                for key, column in mapping.items():
                    mapped_record[column] = record.get(key, None)  # Utiliser None si la clé est absente
                
                # Valider avec Pydantic
                validated_record = Record(**mapped_record)
                valid_records.append(validated_record.model_dump())
                
            except ValidationError:
                # Si la validation échoue (par exemple, 'nom' n'est pas une chaîne), enregistrer dans les invalides
                invalid_records.append(record)

except FileNotFoundError:
    print("Le fichier data.cdr n'a pas été trouvé.")
    exit(1)

# Créer un DataFrame avec les enregistrements valides
df = pd.DataFrame(valid_records)

# Enregistrer les enregistrements invalides dans mauvaise_structure.cdr
with open('mauvaise_structure.cdr', 'w') as file:
    for record in invalid_records:
        json.dump(record, file)
        file.write('\n')

In [ ]:
import json
import pandas as pd
from pydantic import BaseModel, ValidationError
from typing import Optional
import uuid

# Définition du modèle Pydantic pour valider les données
class Record(BaseModel):
    date: str  # Champ obligatoire
    nom: str   # Champ obligatoire
    ville: Optional[str]
    code: Optional[str]

    @field_validator('code')
    def code_must_start_with_qt(cls, value: Optional[str]) -> Optional[str]:
        if value is not None and not value.startswith('qt'):
            raise ValueError('Le code doit commencer par "qt"')
        return value


# Dictionnaire de mappage
mapping = {'1': 'date', '2': 'nom', '3': 'ville', '4': 'code'}

# Listes pour stocker les données valides et invalides
valid_records = []
invalid_records = []

# Lire le fichier .cdr
try:

    #######################si il est deja sous forme de liste json3##################
    with open('sample copy.cdr', 'r') as file:
         # Lire tout le contenu du fichier
        content = file.read().strip()
        records_list = json.loads(content)

    ########################### SINON ####################################################
        # Charger la liste JSON
    # with open('sample copy.cdr', 'r') as f:
    #     cdr_content = f.read()
    #     json_content = f'[{cdr_content}]'
    #     json_list = json.loads(json_content)
    #     datas = json.dumps(json_list, indent=4)
                
    #     records_list = json.loads(datas)
        
        # Traiter chaque enregistrement dans la liste
        for record in records_list:
            try:
                mapped_record = {}
                for key, column in mapping.items():
                    mapped_record[column] = record.get(key, None)  # Utiliser None si la clé est absente
                
                # Valider avec Pydantic
                validated_record = Record(**mapped_record)
                valid_records.append(validated_record.model_dump())
                
            except ValidationError:
                # Si la validation échoue (par exemple, 'nom' n'est pas une chaîne), enregistrer dans les invalides
                invalid_records.append(record)

except FileNotFoundError:
    print("Le fichier sample.cdr n'a pas été trouvé.")
    exit(1)
except json.JSONDecodeError as e:
    print(f"Erreur lors du parsing JSON: {e}")
    exit(1)

# Créer un DataFrame avec les enregistrements valides
df = pd.DataFrame(valid_records)

# Enregistrer les enregistrements invalides dans mauvaise_structure.cdr
with open('mauvaise_structure.cdr', 'w') as file:
    for record in invalid_records:
        json.dump(record, file)
        file.write('\n')
